Import Dependencies

In [29]:
import requests
import time
import json
from unidiff import PatchSet

Set API token and repo URL

In [30]:
API_TOKEN = "api-dja5fvwnsw32z36t56vrbuz4en6t"
BASE_URL = "https://phabricator.services.mozilla.com/api/"

Get all the repositories

In [ ]:
operation = "diffusion.repository.search"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
}

requests.post(API_URL, data=payload).json()

Get revision for a bug

In [22]:
operation = "differential.revision.search"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "constraints[query]": "1931487"
}

requests.post(API_URL, data=payload).json()

{'result': {'data': [{'id': 229239,
    'type': 'DREV',
    'phid': 'PHID-DREV-742ecttyirpscb3cma4q',
    'fields': {'stackGraph': {'PHID-DREV-742ecttyirpscb3cma4q': []},
     'title': 'Bug 1931487 - Add an MruCache for lang-to-script mappings.',
     'uri': 'https://phabricator.services.mozilla.com/D229239',
     'authorPHID': 'PHID-USER-autti2wxdumepz7idru4',
     'status': {'value': 'published',
      'name': 'Closed',
      'closed': True,
      'color.ansi': 'cyan'},
     'repositoryPHID': 'PHID-REPO-saax4qdxlbbhahhp2kg5',
     'diffPHID': 'PHID-DIFF-xh5y22yks5g64gehbgyl',
     'diffID': '946314',
     'summary': '',
     'testPlan': '',
     'isDraft': False,
     'holdAsDraft': False,
     'dateCreated': 1731773380,
     'dateModified': 1731794654,
     'policy': {'view': 'public', 'edit': 'PHID-PROJ-njo5uuqyyq3oijbkhy55'},
     'bugzilla.bug-id': '1931487',
     'uplift.request': []},
    'attachments': {}}],
  'maps': {},
  'query': {'queryKey': None},
  'cursor': {'limit': 10

Get the diff for a revision

In [23]:
operation = "differential.diff.search"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "constraints[revisionPHIDs][0]": "PHID-DREV-742ecttyirpscb3cma4q"
}

requests.post(API_URL, data=payload).json()

{'result': {'data': [{'id': 946314,
    'type': 'DIFF',
    'phid': 'PHID-DIFF-xh5y22yks5g64gehbgyl',
    'fields': {'revisionPHID': 'PHID-DREV-742ecttyirpscb3cma4q',
     'authorPHID': 'PHID-USER-autti2wxdumepz7idru4',
     'repositoryPHID': 'PHID-REPO-saax4qdxlbbhahhp2kg5',
     'refs': [{'type': 'base',
       'identifier': '00a3e992ba4fe62cba6bcf381b98128da0bc581a'}],
     'dateCreated': 1731794384,
     'dateModified': 1731794649,
     'policy': {'view': 'public'}},
    'attachments': {}},
   {'id': 946305,
    'type': 'DIFF',
    'phid': 'PHID-DIFF-fmurav5p5edly32whqho',
    'fields': {'revisionPHID': 'PHID-DREV-742ecttyirpscb3cma4q',
     'authorPHID': 'PHID-USER-autti2wxdumepz7idru4',
     'repositoryPHID': 'PHID-REPO-saax4qdxlbbhahhp2kg5',
     'refs': [{'type': 'branch', 'name': 'default'},
      {'type': 'base',
       'identifier': '6c508a387477e3b72db913a9e1761e9a433d06a2'}],
     'dateCreated': 1731773378,
     'dateModified': 1731773380,
     'policy': {'view': 'public'}

Get raw diff

In [ ]:
operation = "differential.getrawdiff"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "diffID": "946305"
}

requests.post(API_URL, data=payload).json()

Format diff for better legibility

In [31]:
operation = "differential.getrawdiff"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "diffID": "946305"
}

diff = requests.post(API_URL, data=payload).json()

patch = PatchSet(diff["result"])

for patched_file in patch:
    print(f"📄 {patched_file.path}")
    for hunk in patched_file:
        print(f"  🔢 Hunk: {hunk.source_start}:{hunk.source_length} → {hunk.target_start}:{hunk.target_length}")
        for line in hunk:
            if line.is_added:
                print(f"    ➕ {line.value.strip()}")
            elif line.is_removed:
                print(f"    ➖ {line.value.strip()}")


📄 gfx/thebes/gfxTextRun.cpp
  🔢 Hunk: 21:10 → 21:11
    ➕ #include "mozilla/MruCache.h"
  🔢 Hunk: 2594:10 → 2595:70
    ➕ static Script ResolveScriptForLang(const nsAtom* aLanguage, Script aDefault) {
    ➕ // Cache for lang-to-script lookups, to avoid constantly needing to parse
    ➕ // and resolve the lang code from scratch.
    ➕ class LangScriptCache
    ➕ : public MruCache<const nsAtom*, std::pair<const nsAtom*, Script>,
    ➕ LangScriptCache> {
    ➕ public:
    ➕ static HashNumber Hash(const nsAtom* const& aKey) { return aKey->hash(); }
    ➕ static bool Match(const nsAtom* const& aKey,
    ➕ const std::pair<const nsAtom*, Script>& aValue) {
    ➕ return aKey == aValue.first;
    ➕ }
    ➕ };
    ➕ 
    ➕ static LangScriptCache sCache;
    ➕ static RWLock sLock("LangScriptCache lock");
    ➕ 
    ➕ MOZ_ASSERT(aDefault != Script::INVALID &&
    ➕ aDefault < Script::NUM_SCRIPT_CODES);
    ➕ 
    ➕ {
    ➕ // Try to use a cached value without taking an exclusive lock.
    ➕ AutoRe

In [ ]:
operation = "differential.revision.search"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "constraints[query]": "1931487"
}

requests.post(API_URL, data=payload).json()